# <center> Gender Biais detection</center>
## <center> An approach based on the Caliskan Paper 2016 </center>
### <center> Quillivic Robin </center>
Sources : Semantics derived automatically from language corpora necessarily contain human biases, 2016


In [6]:
from gensim import *
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import logging
import gensim
import os

from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

import matplotlib.pyplot as plt
from random import *

import pandas as pd
import numpy as np
import lib

from sklearn.decomposition import PCA
import itertools

C:\Users\Robin\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## DATA (W2VNEWS)

In [12]:
# Logging code taken from http://rare-technologies.com/word2vec-tutorial/
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format(r'DATA\ENG\GoogleNews-vectors-negative300.bin', binary=True,limit=50000)


2019-02-24 18:24:30,895 : INFO : loading projection weights from DATA\ENG\GoogleNews-vectors-negative300.bin
2019-02-24 18:24:31,604 : INFO : loaded (50000, 300) matrix from DATA\ENG\GoogleNews-vectors-negative300.bin


## 1)  Word Embedding Association Test (WEAT)

On considère alors deux sets de mots d'attributs par exemple (man,boy,brother...) et (woman, daughter,sister,...) et  deux sets de mots cibles (informaticien, docteur, ministre..) et (infirmière, libraire, secrétaire..). L'hypothèse nulle est qu'il n'y a pas de différence entre les deux sets cibles en termes de similarité relative avec les deux sets d'attributs. Nous devons maintenant construire un test statistiques pour cette hypothèse. Soit X,Y deux sets cibles de même taille et A,B deux sets d'attributs:
\begin{equation*}
    S(X,Y,A,B) = \sum_{x \in X}s(x,A,B) -\sum_{y \in Y}s(y,A,B)
\end{equation*}
avec
\begin{equation*}
     s(w,A,B) = moy_{a∈A}cos(w,a) − mow_{b∈B}cos(w,b)
\end{equation*}
En d'autre termes, la grandeur s mesure l'association du mot w avec les attributs A,B. La grandeur S, quand à elle la différences d'association mais entre deux set de mots cibles.

In [13]:
def s(w,A,B,model):
    """
        compute the association of the word w, with the attribute A,B
    entry:
    - w: word type: string 
    - A: attribute 1 type:list
    - B: attribute 2 type:list
    - model: Gensim W2V model
    return type:float
    """
    M=0
    N=0
    for a in A :
        M+=lib.cosine_similarity(model[w],model[a])
    for b in B : 
        N+=lib.cosine_similarity(model[w],model[b])
    return(M/len(A)-N/len(B))

def S(X,Y,A,B,model):
    """
       compute the difference of association for the set X and Y, with the attribute A,B
    entry:
    - X: target 1 type: list(string) 
    - Y: target 2 type: list(string)
    - A: attribute 1 type:list
    - B: attribute 2 type:list
    - model: Gensim W2V model
    return type:float
    """
    M=0
    N=0
    for x in X :
        M+=s(x,A,B,model)
    for y in Y : 
        N+=s(y,A,B,model)
    return(M-N)

def effet_size(X,Y,A,B,model) :
    """
    Compute the effect size of the association S
     entry:
    - X: target 1 type: list(string) 
    - Y: target 2 type: list(string)
    - A: attribute 1 type:list
    - B: attribute 2 type:list
    - model: Gensim W2V model
    return type:float
    """
    M=0
    N=0
    st =[]
    for x in X :
        M+=s(x,A,B,model)
    for y in Y : 
        N+=s(y,A,B,model)
    for u in np.concatenate([X,Y]):
        st.append(s(u,A,B,model))
        
    return((M/len(X)-N/len(Y)/np.std(st)))

def parties_union(X,Y):
    """
    Compute all the all the partitions of X∪Y into two sets of equal size
     entry:
    - X: target 1 type: list(string) 
    - Y: target 2 type: list(string)
    return :
    - f: list of partition type: list(list(string))
    """
    f=list(itertools.combinations(np.concatenate([X,Y]),len(X)))

    for i in range(1,len(X)):
        a= list(itertools.combinations(np.concatenate([X,Y]),i))
        f=f+a
    return(f)

def p_values(X,Y,A,B,model):
    """
    Compute the p value of the partition test S
     entry:
    - X: target 1 type: list(string) 
    - Y: target 2 type: list(string)
    - A: attribute 1 type:list
    - B: attribute 2 type:list
    - model: Gensim W2V model
    return type:float
    """
    P=[]
    a= parties_union(X,Y)
    for x,y in itertools.combinations(a,2) :
        P.append(S(x,y,A,B,model))
    n=np.size(np.where(P>S(X,Y,A,B,model)))
    print(n)
    return(n/len(P))


In [21]:
#attributes
A = ['he','man','male','son','guy','father','boy','brother']
B = ['she','woman','female','gal','mother','girl','sister']

# targets
X = ['programmer', 'engineer', 'scientist','doctor'] 
Y = ['nurse', 'teacher', 'librarian','receptionist']

# targets
X=['mathematics', 'algebra', 'geometry', 'calculus', 'equation','computation', 'numbers', 'addition']
Y =['poetry', 'art', 'dance', 'literature', 'novel', 'symphony', 'drama', 'sculpture']  

In [22]:
for x in X:
    print('Association of',x,'with the attributes A,B is : ',s(x,A,B,model)*100)

Association of mathematics with the attributes A,B is :  0.8980484280203066
Association of algebra with the attributes A,B is :  -2.7348050018190406
Association of geometry with the attributes A,B is :  1.515679231462335
Association of calculus with the attributes A,B is :  1.9804423076233688
Association of equation with the attributes A,B is :  4.614481491444167
Association of computation with the attributes A,B is :  -0.2938093328599021
Association of numbers with the attributes A,B is :  1.7144521781509474
Association of addition with the attributes A,B is :  0.38606865813822605


In [23]:
for x in Y:
    print('Association of',x,'with the attributes A,B is : ',s(x,A,B,model)*100)

Association of poetry with the attributes A,B is :  -4.223012125917844
Association of art with the attributes A,B is :  -3.089501399413815
Association of dance with the attributes A,B is :  -8.699945832735725
Association of literature with the attributes A,B is :  -5.040515215867863
Association of novel with the attributes A,B is :  -4.391218279488385
Association of symphony with the attributes A,B is :  -1.8784300523943136
Association of drama with the attributes A,B is :  -1.1906615990613183
Association of sculpture with the attributes A,B is :  1.3166548684239388


# Comment :
The word from the Group A seems more associated to man (>0) and those in group B more associated to Woman (>0), let's calculate the S value to confirme this intuition.

In [29]:
S(X,Y,A,B,model)

0.35277187596615733

#### S is significant , >0.1, that means we ar eobserving a Biais in the association of the target sets .  

In [94]:
p_values(X,Y,A,B,model)


0


0.0

In [27]:
e = effet_size(X,Y,A,B,model)
print('the effect size : ',e)

the effect size :  1.0512952235528474


 The effet size is supetior to 1  and the p value is small < 10^-5 that means we are really oserving a biais and the S, values is not due to random order')

## 2) Word Embedding Factual Association Test (WEFAT) 
Cette seconde méthode présente comment  à partir d'un système vectoriel de mots, nous pouvons retrouver des statistiques réels. Comme par exemple les occupations principalement occupé par les femmes. Pour ce faire, nous pouvons collecter les noms de métiers les plus rependus, constituer un set d'attribut féminin et un set d'attribut masculin et par les grandeurs présenté plus haut décider si une  profession est plus masculine ou féminine et ainsi reconstituer les métier genrés puis utiliser un test statistique pour comparer nos résultats à la réalité.
Soit W l'ensemble des mots dont on veut calculer l'association au genre et A, B les deux sets d'attribut. On introduit:
\begin{equation*}
    wefat(w,A,B) = \frac{moy_{a\in A}cos(w,a) − moy_{b\in B}cos(w,b) }{std_{w \in A\cup B}(cos(w,A,B))}
\end{equation*}

In [30]:
def wefat(w,A,B,model):
    """
    compute the WEFAT coefficient of the word w, with the attribute A,B
    entry:
    - w: word type: string 
    - A: attribute 1 type:list
    - B: attribute 2 type:list
    - model: Gensim W2V model
    return type:float
    """
    M=0
    N=0
    st=[]
    for a in A :
        M+=lib.cosine_similarity(model[w],model[a])
    for b in B : 
        N+=lib.cosine_similarity(model[w],model[b])
    
    for u in np.concatenate([A,B]):
        st.append(lib.cosine_similarity(model[w],model[u]))
        
    diff_=M/len(A)-N/len(B)
    
    return(diff_/np.std(st))

In [31]:
wefat('engineer',A,B,model)

0.8959936377720886